GET /log.gif?1617134399343&
_ts_id=37063F08390D38003009.1570019781&
type=impress&
nick=9393933737f62736&
ts_set=also_22035204797731_938395_000200120013&
rule=main_also&
shipdisc=1&
reqstid=9a941a21-1148-449e-992e-3d1f8b78d0a7&
position=13&
HTTP/1.1 

.withColumn("recommender", F.regexp_extract("Request",'(ts_set=)(\S+)_(\d{14})_(\d+)_(\d+)', 2)) 
.withColumn("time", F.unix_timestamp(F.regexp_extract("Request",'(log\.gif\?)(\d+)', 2))) 

In [ ]:
#刷新次數人次表
df_tslog.groupBy('requestId').agg(sf.max('row_order').alias('refresh_times'), F.sum('was_impressed').alias('impress_times'))  \
.withColumn('user',lit(1))  \
.groupBy('refresh_times')  \
.agg(F.sum('user').alias('users'),F.sum('impress_times').alias('impress_times') )  \
.sort(asc('refresh_times')).show(10)

In [ ]:
#只看刷新次數
df_refresh = df_tslog.groupBy('requestId','row_order').agg(F.sum('was_clicked').alias('click_times'),F.count('was_clicked').alias('impress_times'))  \
.groupBy('row_order').agg(F.sum('click_times').alias('click_times'), F.sum('impress_times').alias('impress_times'), F.count('requestId').alias('user_num'))  \
.withColumn('CTR',F.col('click_times')/F.col('impress_times'))  \
.withColumn('impress/user', F.col('impress_times')/F.col('user_num'))  \
.withColumn('click/user', F.col('click_times')/F.col('user_num'))
df_refresh.sort(sf.asc('row_order')).show(10)

In [ ]:
#impress/user by row_order

In [ ]:
#刷新一次(row order==1)(row order==2)之CTR線
df_refresh_2 = df_tslog.groupBy('requestId').agg(sf.max('row_order').alias('refresh_times')).filter(F.col('refresh_times')==2)
df_refresh_2 = df_refresh_2.join(df_tslog,['requestId'],how='left')
df_refresh_2 = df_refresh_2.filter(F.col('position')<=100)
df_refresh_2.cache()
df_refresh_2.show(1)

In [ ]:
#還沒刷&刷第一次的CTR差異
df_refresh_2 = df_refresh_2.groupBy('row_order','position','recommender').agg(F.sum('was_impressed').alias('impress_times'), F.sum('was_clicked').alias('clicked_times'))

In [ ]:
%%spark -o df_refresh_2 -n -1

In [ ]:
%%local
df = df_refresh_2.groupBy(['row_order','position']).agg('sum')

In [ ]:
#各個recall 還沒刷&刷第一次在的CTR差異


In [ ]:
#刷新二次
df_refresh_3 = df_tslog.groupBy('requestId').agg(sf.max('row_order').alias('refresh_times')).filter(F.col('refresh_times')==3)
df_refresh_3 = df_refresh_2.join(df_tslog,['requestId'],how='left')
df_refresh_3 = df_refresh_2.filter(F.col('position')<=100)
df_refresh_3.cache()
df_refresh_3.show(1)

In [ ]:
#刷二次的人次
df_refresh_3.groupBy('requestId').count()

In [ ]:
#discount
df_discount = df_tslog.filter(F.col('position')<=100).groupBy(['position','sellerDiscount','rutenDiscount','noShippingCost','pCoin','oversea','shippingDiscount','campaign','limitesDeliver'])  \
.agg(F.sum('was_clicked').alias('click_times'),F.count('was_clicked').alias('impress_times'))

In [ ]:
%%spark -o discount -n -1

In [ ]:
%%local
discount_list = ['sellerDiscount','rutenDiscount','noShippingCost','pCoin','oversea','shippingDiscount','campaign','limitesDeliver']
fig , ax = plt.subplots()
fig.subplots_adjust(hspace=0.4, wspace=0.4) #設定子圖的間隔
for i, discount in enumerate(discount_list):
    df = df_discount.groupby(['position',discount]).agg({"click_times":"sum","impress_times":"sum"})
    df['CTR'] = df[('click_times','sum')] / df[('impress_times','sum')] 
    df = df.sort_values('position')
    plt.subplot(3, 3, i)
    plt.plot(df[df[discount]==0]['position'], df[df[discount]==0]['CTR'], color = 'r', label='0')
    plt.plot(df[df[discount]==1]['position'], df[df[discount]==1]['CTR'], color = 'b', label='1')
    plt.set_ticklabels([])
    plt.legend()